In [22]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]    
    # 바이어스 때문에 n+1
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)


# input 2, hidden 1, output 1 -> 바이어스 추가해서 3,2,2

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [23]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
       
        activation += weights[i]*inputs[i]      # 뒤에 바이어스 빼고 weights와 input 인자를 곱함 
        # 순전파 진행 - 퍼셉트론과 동일하게 타우(s), s = x*w
    return activation

def sigmoid(activation):
    return 1/(1+np.exp(-activation)) # 시그모이드 구현

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요? 활성함수 시그모이드를 지나옴
            
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
        inputs = new_inputs
    return inputs

In [24]:
network[0][0]

{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}

## 순전파(forward propagation)
nn 모델의 입력층부터 출력층까지 순서대로 변수들들 계산하고 저장하는 것


In [26]:
seed(1)
network = initialize_network(2, 1, 2)
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [27]:
# 가중치 변화 확인
seed(1)
test_network = initialize_network(2,1,2)
test_network     # 초기 가중치

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'weights': [0.2550690257394217, 0.49543508709194095]},
  {'weights': [0.4494910647887381, 0.651592972722763]}]]

In [28]:
seed(1)
test_network = initialize_network(2,1,2)[0]
network     # 순전파 학습 후 가중치 -> 변하지 않음.

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614],
   'output': 0.7105668883115941}],
 [{'weights': [0.2550690257394217, 0.49543508709194095],
   'output': 0.6629970129852887},
  {'weights': [0.4494910647887381, 0.651592972722763],
   'output': 0.7253160725279748}]]

In [29]:
seed(1)
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


## 역전파(back propagation)
nn의 파라미터들에 대한 그래디언트를 계산하는 방법.
- 손실loss와 정규화 항목s에 대해서 목적함수 J = L + s의 그래디언트 계산
- 출력층 o변수들에 대한 목적함수의 그래디언트를 체인룰 적용해서 구함.
- 출력층과 가장 가까운 모델 파라미터들에 대해서 목적함수의 그래디언트 계산 가능.
- https://ko.d2l.ai/chapter_deep-learning-basics/backprop.html

In [30]:
def sigmoid_derivative(output):
    return output * (1-output) # 시그모이드 미분 y(1-y)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else:
            for j in range(len(layer)):     # 처음 출력값 전 레이어
                neuron = layer[j]
                errors.append(expected[j]-neuron['output']) # 역전파시 오차는 어떻게 설정했나요?
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = (errors[j])*sigmoid_derivative(neuron['output']) 
            # 시그모이드 함수를 사용한 역전파
            # 델타 = (y-o)*타우미분

# for문이 reversed함수 때문에 for -> else -> if 순서로 역전되서 돌아감.


In [31]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [32]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [33]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j] # 퍼셉트론 학습 규칙-델타규칙
            neuron['weights'][-1] += l_rate * neuron['delta'] # 바이어스는 input을 1로
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network,row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # 예측값의 오차 합
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [34]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [35]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [44]:
n_inputs = len(dataset[0])-1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
n_outputs = 2 # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 100, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))


>epoch=0, lrate=0.500, error=6.414
>epoch=1, lrate=0.500, error=5.672
>epoch=2, lrate=0.500, error=5.322
>epoch=3, lrate=0.500, error=5.290
>epoch=4, lrate=0.500, error=5.317
>epoch=5, lrate=0.500, error=5.336
>epoch=6, lrate=0.500, error=5.344
>epoch=7, lrate=0.500, error=5.346
>epoch=8, lrate=0.500, error=5.345
>epoch=9, lrate=0.500, error=5.341
>epoch=10, lrate=0.500, error=5.336
>epoch=11, lrate=0.500, error=5.329
>epoch=12, lrate=0.500, error=5.320
>epoch=13, lrate=0.500, error=5.308
>epoch=14, lrate=0.500, error=5.293
>epoch=15, lrate=0.500, error=5.270
>epoch=16, lrate=0.500, error=5.237
>epoch=17, lrate=0.500, error=5.187
>epoch=18, lrate=0.500, error=5.107
>epoch=19, lrate=0.500, error=4.985
>epoch=20, lrate=0.500, error=4.814
>epoch=21, lrate=0.500, error=4.592
>epoch=22, lrate=0.500, error=4.308
>epoch=23, lrate=0.500, error=3.949
>epoch=24, lrate=0.500, error=3.542
>epoch=25, lrate=0.500, error=3.143
>epoch=26, lrate=0.500, error=2.763
>epoch=27, lrate=0.500, error=2.410
>e

In [45]:
# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))
for layer in network:
    print(layer)

[{'weights': [-1.3341665869601682, 1.7507045557940508, 1.1248841983420608], 'output': 0.05039717066028191, 'delta': -0.0009590275195105367}, {'weights': [1.6566837629670637, -2.494145776285515, -0.6463864214038534], 'output': 0.9637491937143909, 'delta': 0.0009796655350391295}]
[{'weights': [2.6164708539696493, -3.358089232102404, 0.4467736751054617], 'output': 0.06575403376730371, 'delta': -0.004039299279402648}, {'weights': [-2.3589742557405047, 3.6101498875859224, -0.6992643229307567], 'output': 0.93445659065289, 'delta': 0.0040143680526277285}]


In [64]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요? 아웃풋을 최대화 시키는 곳의 위치반환 0아니면 1임.

In [62]:
# forward_propagate(network, row)

[0.0650054833233608, 0.935194562007408]

In [65]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network,row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 

    print('실제값=%d, 예측값=%d' % (row[-1], prediction)) # row 마지막이 타켓, 실제값


실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
